## Similaridade Semantica
Localizar dentro de um texto oferecido a similaridade com uma frase pré determinada

In [1]:
import tensorflow_hub as hub
import tensorflow_text  
import tensorflow as tf
import numpy as np
import re
from transformers import AutoTokenizer, TFAutoModel
from sklearn.metrics.pairwise import cosine_similarity

# LaBSE Embbeding

In [2]:
# Carrega tokenizer e modelo LaBSE
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
model = TFAutoModel.from_pretrained("sentence-transformers/LaBSE")

2025-09-02 18:33:09.022921: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-09-02 18:33:09.023076: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-02 18:33:09.023082: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
I0000 00:00:1756848789.023580 7144479 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1756848789.023632 7144479 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: [

In [3]:
# Função para tokenizar e preparar inputs para o modelo LaBSE
def preprocess(texts):
    tokens = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="tf"
    )
    return {
        "input_ids": tokens["input_ids"],
        "input_mask": tokens["attention_mask"],
        "segment_ids": tf.zeros_like(tokens["input_ids"]) 
    }


In [4]:
# Frases base
texts = [
    "O produto chegou quebrado.",
    "O atendimento foi excelente.",
    "Entrega rápida e segura.",
    "Não gostei da qualidade.",
    "O item está com defeito."
]

In [5]:
# Frase de busca (query)
queries = ["produto com defeito", 
         "chegou quebrado"]

In [8]:
# Tokenização
text_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="tf")
query_inputs = tokenizer(queries, padding=True, truncation=True, return_tensors="tf")

# Geração dos embeddings
#outputs = model(inputs)
text_embeddings = model(**text_inputs).last_hidden_state[:, 0, :]  # [CLS]
query_embeddings = model(**query_inputs).last_hidden_state[:, 0, :]  # [CLS]

# Similaridade de cosseno
similarity_matrix = cosine_similarity(query_embeddings.numpy(), text_embeddings.numpy())

# Exibir resultados
for i, query in enumerate(queries):
    scores = similarity_matrix[i]
    idx = np.argmax(scores)
    print(f"\nQuery: \"{query}\"")
    print(f"Frase mais similar: \"{texts[idx]}\"")
    print(f"Score: {scores[idx]:.4f}")


Query: "produto com defeito"
Frase mais similar: "O item está com defeito."
Score: 0.7929

Query: "chegou quebrado"
Frase mais similar: "O produto chegou quebrado."
Score: 0.6929


## Cria uma função para ser aplicável a diferentes tipos de inteção 

In [13]:
# Função de similaridade
def buscar_similares(texts, queries, categoria=None, top_n=1, threshold=0.5):
    # Tokeniza
    text_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="tf")
    query_inputs = tokenizer(queries, padding=True, truncation=True, return_tensors="tf")

    # Embeddings
    text_embeddings = model(**text_inputs).last_hidden_state[:, 0, :]
    query_embeddings = model(**query_inputs).last_hidden_state[:, 0, :]

    # Similaridade
    similarity_matrix = cosine_similarity(query_embeddings.numpy(), text_embeddings.numpy())

    # Resultados por query
    resultados = []
    for i, query in enumerate(queries):
        scores = similarity_matrix[i]
        top_indices = scores.argsort()[::-1][:top_n]
        top_matches = [
            {
                "query": query,
                "categoria": categoria,
                "texto_encontrado": texts[idx],
                "score": float(scores[idx])
            }
            for idx in top_indices if scores[idx] >= threshold
        ]
        resultados.extend(top_matches)

    return resultados

In [15]:
texts = [
    "Cobra o valor de R$ 1.250,00 em 8x.",
    "Cobra R$10,00 pelo produto.",
    "Cobrar dez reais.",
    "Cobre dez conto",
    "Cobra um valor de 1 real e cinquenta centavos",
    "Como está o tempo hoje?"
]

In [18]:
queries = [
    "Cobra um valor",
     "Cobra o valor",
    "Faça uma cobrança",
    "Passa uma cobrança de dez reais",
    "Cobra dele",
    "Cobre isso aqui",
    "Cobrar o valor restante do cliente"
]
 
# Precisão acima de 65% 
resultados = buscar_similares(texts, queries, categoria="intenção de cobrança", threshold=0.65)
for r in resultados:
    print(f"{r}\n")

{'query': 'Cobra um valor', 'categoria': 'intenção de cobrança', 'texto_encontrado': 'Cobra um valor de 1 real e cinquenta centavos', 'score': 0.6940731406211853}

{'query': 'Cobra o valor', 'categoria': 'intenção de cobrança', 'texto_encontrado': 'Cobre dez conto', 'score': 0.6896507740020752}

{'query': 'Passa uma cobrança de dez reais', 'categoria': 'intenção de cobrança', 'texto_encontrado': 'Cobrar dez reais.', 'score': 0.8900354504585266}

{'query': 'Cobra dele', 'categoria': 'intenção de cobrança', 'texto_encontrado': 'Cobre dez conto', 'score': 0.6705611348152161}

